<a href="https://colab.research.google.com/github/SergeyShchus/Automatic_GIS/blob/master/Notebook/Geopandas/Create_map_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [3]:
import geopandas as gpd, folium, requests, pandas as pd, fiona, branca, json, numpy as np

In [4]:
income = pd.read_csv("https://raw.githubusercontent.com/pri-data/50-states/master/data/income-counties-states-national.csv")
income.columns

Index(['fips', 'county', 'state', 'income-2015', 'income-1989a',
       'income-1989b', 'change'],
      dtype='object')

In [5]:
gdf = gpd.read_file("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json")
statesabbr=pd.read_json("https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_titlecase.json")
income = pd.read_csv("https://raw.githubusercontent.com/pri-data/50-states/master/data/income-counties-states-national.csv")
income['income-2015']=pd.to_numeric(income['income-2015'], errors='coerce')
income=income.groupby(by=['state'], as_index=False).median()
income['State']=pd.merge(income, statesabbr, left_on='state', right_on='abbreviation')['name']
income.head()
# income.dropna(subset=['income-2015'])[['state','income-2015']].groupby(by='state')

,state,fips,income-2015,income-1989b,change,State
0,AK,2154.0,62561.5,66619.55885,14.758367,Alaska
1,AL,1066.0,38721.5,38644.47667,2.779114,Alabama
2,AR,5074.0,37890.0,34699.07158,10.897394,Arkansas
3,AZ,4012.5,43810.0,39230.65642,NaN,Arizona
4,CA,6057.0,53341.0,50230.52704,6.716596,California


In [6]:
gdf1=gdf.merge(income, left_on='name', right_on='State', how='left').fillna(0)

In [8]:
#gdf1.crs = fiona.crs.from_epsg(4326)
gdf1.head()

,id,name,density,geometry,state,fips,income-2015,income-1989b,change,State
0,01,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",AL,1066.0,38721.5,38644.47667,2.779114,Alabama
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",AK,2154.0,62561.5,66619.55885,14.758367,Alaska
2,04,Arizona,57.050,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",AZ,4012.5,43810.0,39230.65642,0.000000,Arizona
3,05,Arkansas,56.430,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",AR,5074.0,37890.0,34699.07158,10.897394,Arkansas
4,06,California,241.700,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",CA,6057.0,53341.0,50230.52704,6.716596,California


In [9]:
gdf1=gdf1[gdf1['name']!='District of Columbia']

In [14]:
roads = gpd.read_file(r"/content/tl_2016_us_primaryroads.shp")
roads = roads.to_crs(gdf1.crs)
interstates = roads.loc[roads['RTTYP']=='I']
interstates.loc[:,'geometry']=interstates.geometry.simplify(tolerance=0.01, preserve_topology=False)
interstates = interstates.dissolve(by='FULLNAME', aggfunc='first', as_index=False)
interstates.loc[:,'FULLNAME'] = interstates['FULLNAME'].str.replace('- ','-')
interstates.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,FULLNAME,geometry,LINEARID,RTTYP,MTFCC
0,E I-10,"MULTILINESTRING ((-97.69777 29.64504, -97.6673...",1105261674551,I,S1100
1,E I-20,"LINESTRING (-98.97077 32.37467, -98.92656 32.3...",110206001615,I,S1100
2,E I-30,"MULTILINESTRING ((-96.58237 32.84551, -96.5376...",1104487239695,I,S1100
3,E I-96,"MULTILINESTRING ((-84.83710 42.82701, -84.8370...",110467146892,I,S1100
4,I--375,"LINESTRING (-82.65380 27.77718, -82.65581 27.7...",1105083070005,I,S1100


In [15]:
variable = 'income-2015'
gdf1=gdf1.sort_values(by=variable, ascending=True)
colormap = folium.LinearColormap(colors=["red","orange","yellow","green"],vmin=gdf1.loc[gdf1[variable]>0, variable].min(), 
                                 vmax=gdf1.loc[gdf1[variable]>0, variable].max()).to_step(n=5)
colormap.caption = "2015 US Median Household Income"

In [17]:
centroid=gdf1.geometry.centroid

m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=4, tiles='stamentoner')

folium.GeoJson(gdf1[['geometry','name',variable]],
               name="United States of America",
               style_function=lambda x: {"weight":2, 'color':'black','fillColor':colormap(x['properties'][variable]), 'fillOpacity':0.2},
              highlight_function=lambda x: {'weight':3, 'color':'black'},
               smooth_factor=2.0,
              tooltip=folium.features.Tooltip(fields=['name',variable,],
                                              aliases=['State','2015 Median Income'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True)
              ).add_to(m)


folium.GeoJson(interstates,
               name='US Interstate Highways',
               style_function=lambda x: {'weight':2,'color':'#5BDAE9','dashArray':'1 3'},
               highlight_function=lambda x: {'weight':5,'color':'yellow', 'dashArray':'1 7'},
               tooltip=folium.features.Tooltip(fields=['FULLNAME',],labels=False, toLocaleString=True),
               smooth_factor=5.0,
              ).add_to(m)


colormap.add_to(m)

folium.LayerControl(autoZIndex=False, collapsed=False).add_to(m)

m

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


TypeError: ignored